In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# SparkSession
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import to_date

# JDBC
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://securit

In [4]:
# Spark
spark = SparkSession.builder.appName("AWS-Amazon-Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Get files
url = "https://s3://amazongamereview/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [6]:
df.dtypes
# Types look fine

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:
# Setup review dataframe
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()
print(review_id_df.dtypes)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
|R3CFKLIZ0I2KOB|   22151364|B00IMIL498|     621922192| 2015-08-31|
|R1LRYU1V0T3O38|   38426028|B00S00IJH8|     215163395| 2015-08-31|
| R44QKV6FE5CJ2|    6057518|B004RMK4BC|     384246568| 2015-08-31|
|R2TX1KLPXXXNYS|   20715661|B00K59HKIQ|     384246568| 2015-08-31|
|R1JEEW4C6R89BA|   26540306|B00K59HKIQ|     384246568| 2015-08-31|
|R3B3UHK1FO0ERS|    8926809|B004774IPU|     151985175| 2015-08-31|
|R2GVSDHW513SS1|   31525534|B002LIT9EC|     695277014| 2015-08

In [10]:
# Setup products dataframe
products_df = df.select(["product_id", "product_title"])
products_df.show()
print(products_df.count())

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B013PURRZW|Madden NFL 16 - X...|
|B00F4CEHNK| Xbox Live Gift Card|
|B00DNHLFQA|Command & Conquer...|
|B004RMK5QG|Playstation Plus ...|
|B00G9BNLQE|Saints Row IV - E...|
|B00IMIL498|Double Dragon: Ne...|
|B00S00IJH8|              Sims 4|
|B004RMK4BC|Playstation Netwo...|
|B00K59HKIQ|Playstation Netwo...|
|B00K59HKIQ|Playstation Netwo...|
|B004774IPU|Sid Meier's Civil...|
|B002LIT9EC|Build-a-lot 4: Po...|
|B008ALUBYQ|       Borderlands 2|
|B010KYDNDG|Minecraft for PC/...|
|B008ALUBYQ|       Borderlands 2|
|B004RMK4P8|Playstation Netwo...|
|B004RMK5QG|Playstation Plus ...|
|B00BI16Z22|Brink of Consciou...|
|B004VSTQ2A|Xbox Live Subscri...|
|B00GAC1D2G|Playstation Netwo...|
+----------+--------------------+
only showing top 20 rows

145431


In [12]:
# Cleanup
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()
print(products_df.count())
print(products_df.dtypes)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001TK3VTC|Cubis Gold 2 [Dow...|
|B002MUB0TG|Little Shop of Tr...|
|B00DRO824C|X2 The Threat [On...|
|B00CMDKNHI| 100% Hidden Objects|
|B00CMSCTA2|Age of Empires II...|
|B00GDHD3LW|1 Month Subscript...|
|B00OM1VVJM|            NBa 2K15|
|B005XOJ6NK|   Sonic Generations|
|B00GGUHGGY|Call of Duty Blac...|
|B00A9XQ890|Grand Theft Auto:...|
|B003YMMGEE|Sniper: Ghost War...|
|B00405RA04|Rome: Total War -...|
|B0083KX9IO|Stronghold Kingdo...|
|B00M0W1YZQ|Chronicles of Wit...|
|B00895ARJC|  Syberia [Download]|
|B0084FHQ1O|The Stronghold Co...|
|B00KK9PR90|Freddi Fish 3: Th...|
|B001KC01OW|Wonderlines [Down...|
|B008TTPTOQ|Battlefield Bad C...|
|B00DGZDGUM|Nascar The Game 2...|
+----------+--------------------+
only showing top 20 rows

7948
[('product_id', 'string'), ('product_title', 'string')]


In [16]:
# Customer dataframe
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()
print(customers_df.dtypes)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   26079415|             1|
|   12521601|             2|
|    4593696|             1|
|    1468259|             1|
|   38173350|             1|
|    3953141|             2|
|   20598424|             1|
|   46625306|             1|
|   47160828|             2|
|    1337034|             1|
|   14231016|             1|
|   49785748|             1|
|     712525|             1|
|   35339694|             1|
|    8211293|             1|
|   41832251|             1|
|    9037272|             1|
|   52036002|             1|
|   49514578|             1|
|   22875949|             1|
+-----------+--------------+
only showing top 20 rows

[('customer_id', 'int'), ('customer_count', 'bigint')]


In [18]:
# Vine dataframe
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", 'verified_purchase'])
vine_df.show()
print(vine_df.dtypes)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [19]:
from config import aws_db_conn, aws_username, aws_password
# Configure settings for RDS
mode = "append"
jdbc_url="aws_db_conn"
config = {"user":"aws_username", 
          "password": "aws_password", 
          "driver":"org.postgresql.Driver"}

In [25]:
# Write review_id_df to the table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
# Write products_df to the table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)
# Write customers_df to the table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)
# Write vine_df to the table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)